# PV system data filtering test function

In [3]:
"""
filtering functions for cleaning datasets. Specifically for pv applications.
"""

'\nfiltering functions for cleaning datasets. Specifically for pv applications.\n'

In [4]:
from importlib import reload
import pandas as pd
import numpy as np
from datetime import datetime
import datatools
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime
import pvlib
import pvanalytics
import missingno as msno

%matplotlib inline 
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

mpl.rcParams['font.size']=12
mpl.rcParams['lines.linewidth']=1
mpl.rcParams['xtick.labelsize']=10
#mpl.rcParams['font.weight']='bold'
mpl.rcParams['axes.titlesize']=22

### use database values

In [5]:
#import a database using datatools
def import_database(tablename, start, end, save=False):
    engine = datatools.database.create_mss_engine()

    sys_sql = f"select * from {tablename} where TmStamp between '{start}' and '{end}';"
    #sys_sql = f"select * from {tablename}"
    sys = pd.read_sql(sys_sql, engine)#, index_col='TmStamp')
    sys.TmStamp = pd.to_datetime(sys.TmStamp)
    sys = sys.set_index('TmStamp')
    sys.index = sys.index.tz_localize('MST') # pvlib requires indexes to be timezone aware or it assumes UTC
    print([sys.index.min(),  sys.index.max()], [sys.index.max()-sys.index.min()])
    
    if save==True:
        sys.to_csv(f'{tablename}_{start}_{end}.csv')
    
    return sys

### use csv values

before this there should be a way to get the database to save a csv file if you want to import. Or there should be a cleaned csv file that has filters.

In [6]:
#cd C:\Users\jtmcmul\Desktop\Jupyter_Notebook\tools\datatools-master\System CSVs

In [7]:
#This will read a csv file on your computer so you don't have to download data from the database
def file_reader(name):
    """
    Be sure to set your directory path to where the file is located on your computer.
    """
    
    sys = pd.read_csv(name)
    sys.TmStamp = pd.to_datetime(sys.TmStamp)
    sys['TS'] = sys.TmStamp
    sys = sys.set_index('TmStamp')
    [sys.index.min(),  sys.index.max()]
    sys.name = name
    
    return sys

# simple data
only uses important columns during development phase. drops unnescessary columns

In [8]:
#this should drop stuff based on universal naming later on
#Redo this so that it only keeps the names that you want. This makes it more usable for other data
def drop_unnescessary_columns(df):
    df = df.drop(['POACleanRC_E_Avg', 'GHI_Avg', 'Albedo_Avg', 'RecNum',
                  'PH2_V_Avg','PH3_V_Avg','PH4_V_Avg',
                  'PH2_I_Avg','PH3_I_Avg','PH4_I_Avg',
                  'LM1_V_Avg','LM2_V_Avg','LM3_V_Avg','LM4_V_Avg',
                  'LM1_I_Avg','LM2_I_Avg','LM3_I_Avg','LM4_I_Avg',
                  'PH2_RTD_Avg','PH3_RTD_Avg','PH4_RTD_Avg',
                  'LM1_RTD_Avg','LM2_RTD_Avg','LM3_RTD_Avg','LM4_RTD_Avg',
                  'Wind_Gust_Max','Wind_Dir_std','Wind_Direction_Avg'], axis=1)
    return df

In [9]:
#universal naming, This will rename the column based on the dictionary you use.
#this could be replaced with the rename function since it's pretty straightforward.
def naming(df, dictionary):
    df = df.rename(columns=dictionary)
    
    return df

## metadata tracking

this should be implemented into every filter function

In [10]:
def meta_col(df):
    #add a new metadata column
    df['metadata'] = '0'
    df['metadata'].astype(str)
    
    #add the convention as a dictionary
    md = {'Code':[0],
          'Description':['Raw data']}

    #make a new dataframe to hold the names
    df_meta = pd.DataFrame(data=md) 
    
    return df, df_meta

# 1a - Shape and Interval Freq

In [11]:
# find recording interval and reporting period

In [12]:
#these functions should be combined
# find rows and columns

def shape(sys):
    """
    Tells you the number of rows and the number of columns of the dataframe. Uses a Pandas DataFrame as input.
    
    Parameters
    ----------
    sys : df
        Pandas dataframe.
    
    Return
    ------
    rc
        A string telling you the number of rows and columns
    """
    nrows, ncols = sys.shape
    rc = print(f'Number of rows = {nrows}, Number of columns = {ncols}')
    return rc

def interval_freq(sys, plot=True, apply_working_mask=True):
    """
    Tells you the recording interval
    
    Parameters
    ----------
    sys : df
        Pandas dataframe.
    
    t : int
        the resolution of your data. 
    
    Return
    ------
    monitoring_availability
        The fraction of the time that your system was on.
        
    recording_interval
        the interval over which your data exists.
        
    freq
        the frequency of your data.
    """
    initial_no_of_rows = len(sys)
    #finds the frequency of the data by looking at the interval between the first datasets
    freq = pd.infer_freq(sys.index[:10])
    #this will resample the data based on the infered frequency above. 
    #If there are multiple values in the frequency, then the resample will choose the median value.
    sys = sys.resample(freq).median()
    
    no_of_rows = len(sys)
    
    #recording interval is the time between two consecutive intervals
    d = {'T':'Minutes', 'S':'Seconds', 'M':'Months', 'Y':'Years'}
    #replace the string so that it's easier to read
    if any(char.isdigit() for char in freq)==False:
        recording_interval = '1 ' + d[freq[-1]]
    else:
        recording_interval = freq[:-1] + ' ' + d[freq[-1]]
    
    #reporting period is the total amount of time being recorded
    reporting_period = [sys.index[0], sys.index[-1]], sys.index[-1]-sys.index[0]
    
    #fraction of primary data that wasn't recorded. this accounts for nighttime and flashtest values
    working_mask = (sys.Iave < 0.5) & (sys.irradiance >200) | (sys.irradiance<20) | (sys.Iave<0.1)

    sys_ = sys.drop(sys[working_mask].index)
    #add additional mask to sys_ to be filtered by night, do this by sun position
    monitoring_availability_primary = len(sys_)/len(sys.Vave.asfreq('T'))
    
    #this is the fraction of data that is available because of missing time steps
    monitoring_availability_raw = 100*(initial_no_of_rows/no_of_rows)
    
    
    if plot==True:
        #plot a scatter plot that shows where the data was taken out
        plt.scatter(sys_.index, sys_.Iave, s=6, label='original values')
        plt.scatter(sys.index[working_mask], sys.Iave[working_mask], s=6, label='dropped values')
        plt.legend()
        plt.title('Plot of Available Data')
        plt.ylabel('Current')
        plt.xlabel('Time')
        plt.show()
    else:
        pass
    
    
    if apply_working_mask==True:
        #use the working_mask to drop the data that isn't interesting to the problem.
        working_mask = sys.drop(working_mask[working_mask==True].index)

    else:
        pass
    
    return round(monitoring_availability_primary,4)*100, recording_interval, reporting_period, working_mask

# 2a

In [13]:
#other ways you can do the add missing time step is to use sys.reindex and sys.asfreq(freq)

def add_missing_time_steps(df, df_meta, freq, save=False):
    
    df_comp  = df.asfreq(freq)
    
    
    if save==True:
        #Save csv
        df_comp.to_csv(f'{df.index[0]}_to_{df.index[-1]}_{df.name}_COMPLETE.csv')
        
    #Updates the metadata column
    df_comp.metadata = df_comp.metadata.replace(np.nan, '1')
    
    #adds the metadata code to the meta dataframe
    md = {'Code':[1],
          'Description':['Time steps added']}
    
    df_meta_1 = df_meta.append(pd.DataFrame(data=md), ignore_index=True, )    
    
    return df_comp, df_meta_1
   

In [14]:
def report_reader(x):
    for i in x: print(i)

## Masks

In [15]:
def add_masks(sys):
    
    #create new dataframe to hold masks
    sys_comp_masks = pd.DataFrame(index=sys.index)#, columns = sys.columns[:-1])

    #Create dataframe that describes the type of mask being applied.
    md = {'Mask Name':[np.nan], 'Mask Description':[np.nan]}
    sys_comp_masks_desc = pd.DataFrame(columns=md)

    return sys_comp_masks, sys_comp_masks_desc

incorporate metadata?

# 3a - daylight filter

In [16]:
def daylight_filter_position(sys, sys_masks, sys_masks_desc, sys_meta, 
                             latitude, longitude, degree,
                             apply_mask=True, plot=True):
    #Calculate SunPositions using latitude and longitude
    
    
    solpos = pvlib.solarposition.get_solarposition(sys.index, latitude, longitude)
    dni_extra = pvlib.irradiance.get_extra_radiation(sys.index)
    solpos = solpos.tz_convert('MST')

    #come back to add syntax for .custom degree angle
    #Apply daylight filter if elevation is greater than 10 degrees
    mask_day = (solpos['apparent_elevation']<degree) | (sys.irradiance<0)
    mask_day.name = 'mask_day'
    print(f'Percent of data that is daylight = {round(mask_day.values.mean()*100,3)}%')
    min_POA = min(sys.loc[~mask_day,'irradiance'].values)
    print(f'Minimum POA irradiance during daylight = {round(min_POA,3)} W/m^2')

    #Add to mask dataframe (df_masks)
    sys_masks['mask_day_position']=mask_day

    #Document mask description
    md = {'Mask Name':['mask_day_position'], 'Mask Description':['Apparent elevation > 10']}
    sys_masks_desc = sys_masks_desc.append(pd.DataFrame(data=md, index=[0]))


    #update metadata
    datelist = mask_day.loc[mask_day==False].index
    sys.loc[datelist,'metadata'] = sys.loc[datelist,'metadata']+', 2.1'
    md = {'Code':[2.1],
          'Description':['Daylight filter: Sun Position']}

    sys_meta_2 = sys_meta.append(pd.DataFrame(data=md), ignore_index=True )
    
    #optional plotting function
    if plot==True:
        sys_ = sys.drop(sys[mask_day].index)
        #tell how much data is lost
        print('sys:',len(sys))
        print('sys with filter:',len(sys_))
        print('remaining fraction of usable data:', round(len(sys_)/len(sys)*100,3),'%. Dropped points:', np.abs(len(sys_)-len(sys)))
        #plot a scatter plot that shows where the data was taken out
        plt.scatter(sys_.index, sys_.irradiance, color='blue')
        plt.scatter(sys.index[mask_day], sys.irradiance[mask_day], color='orange')
        plt.xlabel('Time')
        plt.ylabel('Irradiance W/m2')
        plt.show()
    else:
        pass
    
    #this will apply the mask and drop the mask_day values that are True
    if apply_mask==True:
        sys_masks = sys_masks.drop(sys[mask_day].index)
        sys = sys.drop(sys[mask_day].index)
    else:
        pass
    
    
    return sys, sys_masks, sys_masks_desc, sys_meta_2

def daylight_filter_irradiance(sys, sys_masks, sys_masks_desc, sys_meta,
                               apply_mask=True, plot=True):
    #day time if irradiance is greater than 20 w/m2
    mask_day = sys['irradiance']>20
    mask_day.name = 'mask_day'
    print(f'Percent of data that is daylight = {round(mask_day.values.mean()*100,3)}%')
    min_POA = min(sys.loc[mask_day,'irradiance'].values)
    print(f'Minimum POA irradiance during daylight = {round(min_POA,3)} W/m^2')

    #Add to mask dataframe (df_masks)
    sys_masks['mask_day_irradiance']=mask_day

    #Document mask description
    md = {'Mask Name':['mask_day_irradiance'], 'Mask Description':['Irradiance > 20']}
    sys_masks_desc = sys_masks_desc.append(pd.DataFrame(data=md, index=[0]))
    sys_masks_desc

    #update metadata
    datelist = mask_day.loc[mask_day==False].index
    sys.loc[datelist,'metadata'] = sys.loc[datelist,'metadata']+', 2.2'
    #metadata reference key update
    md = {'Code':[2.2],
          'Description':['Daylight filter: irradiance']}

    sys_meta_2 = sys_meta.append(pd.DataFrame(data=md), ignore_index=True )
    
    #optional plotting function
    if plot==True:
        sys_ = sys.drop(sys[~mask_day].index)
        #tell how much data is lost
        print('sys:',len(sys))
        print('sys with filter:',len(sys_))
        print('remaining fraction of usable data:', round(len(sys_)/len(sys),3)*100,'%. Dropped points:', np.abs(len(sys_)-len(sys)))
        #plot a scatter plot that shows where the data was taken out
        plt.scatter(sys_.index, sys_.irradiance)
        plt.scatter(sys.index[~mask_day], sys.irradiance[~mask_day])
        plt.xlabel('Time')
        plt.ylabel('Irradiance W/m2')
        plt.show()
    else:
        pass
    
    #this will apply the mask and drop the mask_day values that are True
    if apply_mask==True:
        sys_masks = sys_masks.drop(sys[~mask_day].index)
        sys = sys.drop(sys[~mask_day].index)
    else:
        pass
    
    
    return sys, sys_masks, sys_masks_desc, sys_meta_2

def daylihgt_filter_time():
    pass

# 4a physical filters

outliers are replaced with na values

In [17]:
#Functions to add mask and description to dataframes
# Is there a way to not have to pass the dataframe into the function but rather define a basename and have 
#   the function figure out the right df to work with?  This would avoid errors.

def add_mask_desc(df_masks_desc, mask, description):
    entry = {'Mask Name':[mask.name], 'Mask Description':[description]}
    lastindex = max(df_masks_desc.index, default=0)
    df2 = pd.DataFrame(data=entry, index=[lastindex+1])
    df_new = df_masks_desc.append(df2)
    return df_new

def add_mask_df(df_masks,mask):
    df_masks[mask.name] = mask.values
    return df_masks

In [18]:
#physical limits, give it a column, start, end, and unit

def physical_filter(sys, variable, lower, upper, sys_comp_masks, sys_comp_masks_desc, sys_meta):
    
    variable = sys[variable]
    # Physical Limits  (note: mask = True for data in acceptable ranges)
    mask_low = variable >= lower
    mask_high = variable <= upper
    #mask_nan = ~varible.isnull() #True = data is not NaN  --> Maybe do this check later
    mask = mask_low | mask_high  ### Should add a check if the values are NaNs?  
    mask.name = f'mask_{variable.name}'

    #Add to documentation trail
    sys_comp_masks = add_mask_df(sys_comp_masks,mask)
    #how to drop a column that is already there? if for name in columns == variable.name: ????
    #sys_comp_masks.drop([sys1.poa.name],axis=1)
    sys_comp_masks_desc = add_mask_desc(sys_comp_masks_desc,mask, f'{variable.name} >={lower}. OR {variable.name} <= {upper}.')
    
    
    #metadata
    datelist = mask.loc[mask==False].index
    code = round(sys_meta.iloc[-1,0]) + 1
    sys.loc[datelist,'metadata'] = sys.loc[datelist,'metadata']+f', {code}'
    md = {'Code':[code],
          'Description':[f'Physical filter: {variable.name}']}

    sys_meta_2 = sys_meta.append(pd.DataFrame(data=md), ignore_index=True )
    
 
    
    return sys, sys_comp_masks, sys_comp_masks_desc, sys_meta_2

In [19]:
#comparison with other sensors

In [20]:
#maximum change between successive data points. Give it a column, limit, and unit

def diff_filter(sys, variable,maxchange, sys_comp_masks, sys_comp_masks_desc, sys_meta ):
    
    variable = sys[variable]

    #mask used for the step change in ambient temperature Tamb

    sys[f'{variable.name}_diff'] = abs(variable.diff())

    diff_mask = sys[f'{variable.name}_diff']<=maxchange

    #Add to documentation trail
    sys_comp_masks = add_mask_df(sys_comp_masks,diff_mask)
    sys_comp_masks_desc = add_mask_desc(sys_comp_masks_desc,diff_mask, f'change in {variable.name} <= {maxchange}')
    
    #metadata
    datelist = diff_mask.loc[diff_mask==False].index
    code = round(sys_meta.iloc[-1,0]) + 1
    sys.loc[datelist,'metadata'] = sys.loc[datelist,'metadata']+f', {code}'
    md = {'Code':[code],
          'Description':[f'Step change filter: {variable.name}']}

    sys_meta_2 = sys_meta.append(pd.DataFrame(data=md), ignore_index=True )
   
    return sys, sys_comp_masks, sys_comp_masks_desc, sys_meta_2

### statistical tests

In [31]:
#statistical and comparative tests. Give it a column, standard deviations, 

def stat_sigma_test(data, deviation=3, apply=False):
        
    mean = np.mean(data)
    stdv = np.std(data)


    z = np.abs((data-mean)/stdv)

    mask =  z > deviation #true is real values, false is bad values

    data_nan = data.mask(mask,np.nan)
    
    if apply==True: #this will apply the mask to the data if true
        return data_nan, mask
    else:
    #if apply==False: #this will only return the mask
        return mask

In [22]:
def hampel_filter(input_series, window_size, n_sigmas=3):

    k = 1.4826 # scale factor for Gaussian distribution
    new_series = input_series.copy()

    # helper lambda function 
    MAD = lambda x: np.median(np.abs(x - np.median(x)))
    
    rolling_median = input_series.rolling(window=2*window_size, center=True).median()
    rolling_mad = k * input_series.rolling(window=2*window_size, center=True).apply(MAD)
    diff = np.abs(input_series - rolling_median)

    indices = np.argwhere(np.array(diff > (n_sigmas * rolling_mad))).flatten()
    new_series[indices] = rolling_median[indices]
    
    return new_series, indices

# 4b

search for na, nan, and black cells

In [23]:
def apply_filter(sys, masks, *args):
    
    sys_nan = sys.where(masks.mask_day, np.nan)
    
    #add documentation to word file
    #document = Document()
    #document.add_heading('Document Title', 0)
    
    for var in args:
        sys_nan[f'{var}'] = sys_nan[f'{var}'].where(masks[f'mask_{var}'], np.nan)

        #p = document.add_paragraph('A plain paragraph having some ')
    
     
    
    return sys_nan

# 5a

identify missing data mechanism (MCAR, MAR, NMAR) by using visualization method.

Use the msno library

In [24]:
def missing_visual(df):
    msno.matrix(df,freq='T')


In [25]:
#make a function that applies the mask to the data so it shows which points are being masked in red.
def visual_dots(sys, mask):

    masked_sys = sys[~mask]
    plt.scatter(sys.index, sys, c='#add8e6',s=6, label='data', alpha=0.4)
    plt.scatter(masked_sys.index, masked_sys, c='#da9b86', label='mask')
    plt.xlabel(sys.index.name)
    plt.ylabel(sys.name)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.show()
    return

In [26]:
#visual inspection of scatter plots
def scatter_inspect(df):
    
    for i in df.columns:
        if i == 'metadata':
            continue
        y = df[f'{i}']
        x = df.index
        plt.scatter(x,y, s=4)
        plt.title(f'{i}')
        plt.show()
    

### missing data rates

Identify missing data rate and missingness rate for every recorded field measurement

In [27]:
def missing_rates(sys, variables):
    
    total_length = len(sys.index)
    
    for var in variables:
        print(f'{var} missing rate:', sys[f'{var}'].isna().sum()/total_length*100,'%')

# 6a

if the missing data rate is less than 10%
- Discard the missing period (listwise deletion) or
- Infer the missing measurements for (a) a whole month missing for a yearly performance analysis and (b) providing robust degradation and performance loss rate estimates


if the missing data rate is greater than 10%\
- if meteorlogical data is available then infer the missing data using an empirical model
 - To fill in missing power measurements use SAPM model from PVLIB
 - To fill in missing module temperatures use the SMTM (or the Ross thermal model35 when only GI and Tamb are available)
 
 
- if meteorlogical data is unavailable then use univariate data imputation techniques

In [28]:
# less than 10% of the data is missing


In [29]:
 def simple_filter(sys, variable, lower=None, upper=None, apply=False):

    variable = sys[variable]

# Physical Limits  (note: mask = True for data in acceptable ranges)


    if lower==None:
        mask_low = pd.Series(dtype='float64')
        if upper==None:
            return print('Please specify upper and/or lower limits')
        else:
            mask_high = variable<=upper
            mask = mask_high
    else:
        mask_low = variable >= lower
        if upper==None:
            mask_high = pd.Series(dtype='float64')
            mask = mask_low
        else:
            mask_high = variable<= upper
            mask = mask_low & mask_high
        
### Should add a check if the values are NaNs?  
    mask.name = f'mask_{variable.name}'

    if apply==True:
        sys = sys.where(mask, np.nan)
        return sys
    else:
        return mask